In [1]:
# pip install -q tensorflow-model-optimization

In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import tempfile
import zipfile
import os

Train a tf.keras model for MNIST without clustering

In [3]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)

Epoch 1/10
1688/1688 [==============================] - 20s 11ms/step - loss: 0.3039 - accuracy: 0.9151 - val_loss: 0.1323 - val_accuracy: 0.9648
Epoch 2/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1320 - accuracy: 0.9621 - val_loss: 0.0933 - val_accuracy: 0.9742
Epoch 3/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.0919 - accuracy: 0.9733 - val_loss: 0.0797 - val_accuracy: 0.9778
Epoch 4/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.0735 - accuracy: 0.9793 - val_loss: 0.0701 - val_accuracy: 0.9817
Epoch 5/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0615 - accuracy: 0.9818 - val_loss: 0.0650 - val_accuracy: 0.9817
Epoch 6/10
1688/1688 [==============================] - 22s 13ms/step - loss: 0.0537 - accuracy: 0.9841 - val_loss: 0.0609 - val_accuracy: 0.9833
Epoch 7/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0481 - accuracy: 0.9857 - val_loss: 0.0597 -

Evaluate the baseline model and save it for later usage

In [4]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.9815000295639038
Saving model to:  /var/folders/gb/4t3qb3rn74q3j_03hdn28vwr0000gn/T/tmp4ek92y82.h5


Define the model and apply the clustering API

In [5]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cluster_reshape (ClusterWei  (None, 28, 28, 1)        0         
 ghts)                                                           
                                                                 
 cluster_conv2d (ClusterWeig  (None, 26, 26, 12)       244       
 hts)                                                            
                                                                 
 cluster_max_pooling2d (Clus  (None, 13, 13, 12)       0         
 terWeights)                                                     
                                                                 
 cluster_flatten (ClusterWei  (None, 2028)             0         
 ghts)                                                           
                                                                 
 cluster_dense (ClusterWeigh  (None, 10)               4

In [6]:
# Fine-tune model
clustered_model.fit(
  train_images,
  train_labels,
  batch_size=500,
  epochs=1,
  validation_split=0.1)

108/108 [==============================] - 12s 102ms/step - loss: 0.0354 - accuracy: 0.9893 - val_loss: 0.0628 - val_accuracy: 0.9855


In [7]:
# Minimal Test Accuracy
_, clustered_model_accuracy = clustered_model.evaluate(
  test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

Baseline test accuracy: 0.9815000295639038
Clustered test accuracy: 0.9800000190734863


**Create 6x smaller models from clustering**

In [8]:
# Create a compressible model for TensorFlow
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file, 
                           include_optimizer=False)

Saving clustered model to:  /var/folders/gb/4t3qb3rn74q3j_03hdn28vwr0000gn/T/tmp_bqqmnrt.h5


In [9]:
# compressable models for TFLite
clustered_tflite_file = '/tmp/clustered_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
    f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)

INFO:tensorflow:Assets written to: /var/folders/gb/4t3qb3rn74q3j_03hdn28vwr0000gn/T/tmpyvhwpvwz/assets


Saved clustered TFLite model to: /tmp/clustered_mnist.tflite


In [10]:
# Define a helper function to actually compress the models via gzip and measure the zipped size
def get_gzipped_model_size(file):
    # It returns the size of the gzipped model in bytes.
    import os
    import zipfile

    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)

In [11]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))

Size of gzipped baseline Keras model: 78213.00 bytes
Size of gzipped clustered Keras model: 12765.00 bytes
Size of gzipped clustered TFlite model: 12347.00 bytes


Create an 8x smaller TFLite model from combining weight clustering and post-training quantization

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

_, quantized_and_clustered_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_clustered_tflite_file, 'wb') as f:
    f.write(tflite_quant_model)

print('Saved quantized and clustered TFLite model to:', quantized_and_clustered_tflite_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_clustered_tflite_file)))

INFO:tensorflow:Assets written to: /var/folders/gb/4t3qb3rn74q3j_03hdn28vwr0000gn/T/tmpl9pzm_8y/assets


INFO:tensorflow:Assets written to: /var/folders/gb/4t3qb3rn74q3j_03hdn28vwr0000gn/T/tmpl9pzm_8y/assets


Saved quantized and clustered TFLite model to: /var/folders/gb/4t3qb3rn74q3j_03hdn28vwr0000gn/T/tmpq5ucop6t.tflite
Size of gzipped baseline Keras model: 78213.00 bytes
Size of gzipped clustered and quantized TFlite model: 9466.00 bytes
